In [223]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce,api
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [224]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(6756, 7)

In [225]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6745, 5)

In [226]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(6762, 9)

fin empresas y comienzo de rescate de pacientes


In [227]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1599396, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [228]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña"        
    }
)
Paciente_Programas_SF.shape

(1160286, 4)

In [229]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1160286, 12)

In [ ]:
empresas_unidas.head()

In [ ]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [ ]:
#pacientes_empresas22.shape

In [ ]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [ ]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [ ]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [230]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Sintec"
Pais = "México"


Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,empl,rut1,sexo,nombres,apellidos,nacimiento,gth,fghfjg,fj,fgjfgj,division,email,rut
0,SINTEC,LUGA810606HDFNMB01,M,Abdon,Luna Gomez,1981-06-06,6,6,1981,1981-06-06,CDMX,abdon.luna@sintec.com,LUGA810606HDFNMB01
1,SINTEC,ROAA900309MVZDGD07,F,Adriana,Rodriguez Aguilar,1990-03-09,9,3,1990,1990-03-09,MTY,adriana.rodriguez@sintec.com,ROAA900309MVZDGD07
2,SINTEC,LOGA970503HNLZRL04,M,Alberto,Lozano Garza,1997-05-03,3,5,1997,1997-05-03,MTY,alberto.lozano@sintec.com,LOGA970503HNLZRL04
3,SINTEC,ZACA890613MDFMRL09,F,Alejandra Berenice,Zamora Cruz,1989-06-13,13,6,1989,1989-06-13,MTY,alejandra.zamora@sintec.com,ZACA890613MDFMRL09
4,SINTEC,AASA961107HNLRLL06,M,Alejandro,Aragon Salazar,1996-11-07,7,11,1996,1996-11-07,MTY,alejandro.aragon@sintec.com,AASA961107HNLRLL06
5,SINTEC,GABA960511HDFRRL06,M,Alejandro,Graf Burgos,1996-05-11,11,5,1996,1996-05-11,CDMX,alejandro.graf@sintec.com,GABA960511HDFRRL06
6,SINTEC,LOGA970321HNLPTL06,M,Alejandro,Lopez Gutierrez,1997-03-21,21,3,1997,1997-03-21,MTY,alejandro.lopez@sintec.com,LOGA970321HNLPTL06
7,SINTEC,MAMA000804HDFRLLA4,M,Alejandro,Martínez Molina,2000-08-04,4,8,2000,2000-08-04,CDMX,alejandro.martinez@sintec.com,MAMA000804HDFRLLA4
8,SINTEC,UICD980624HNLRRV06,M,David Alejandro,Uriarte Correa,1998-06-24,24,6,1998,1998-06-24,MTY,alejandro.uriarte@sintec.com,UICD980624HNLRRV06
9,SINTEC,17493133K,F,Alejandro Daniel,Zepeda Navea,1990-12-07,7,12,1990,1990-12-07,CHI,alejandro.zepeda@sintec.com,17493133K


In [231]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head() 

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16072\836107269.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,empl,rut1,sexo,nombres,apellidos,nacimiento,gth,fghfjg,fj,fgjfgj,division,email,rut
0,SINTEC,LUGA810606HDFNMB01,M,Abdon,Luna Gomez,1981-06-06,6,6,1981,1981-06-06,CDMX,abdon.luna@sintec.com,LUGA810606HDFNMB01
1,SINTEC,ROAA900309MVZDGD07,F,Adriana,Rodriguez Aguilar,1990-03-09,9,3,1990,1990-03-09,MTY,adriana.rodriguez@sintec.com,ROAA900309MVZDGD07
2,SINTEC,LOGA970503HNLZRL04,M,Alberto,Lozano Garza,1997-05-03,3,5,1997,1997-05-03,MTY,alberto.lozano@sintec.com,LOGA970503HNLZRL04
3,SINTEC,ZACA890613MDFMRL09,F,Alejandra Berenice,Zamora Cruz,1989-06-13,13,6,1989,1989-06-13,MTY,alejandra.zamora@sintec.com,ZACA890613MDFMRL09
4,SINTEC,AASA961107HNLRLL06,M,Alejandro,Aragon Salazar,1996-11-07,7,11,1996,1996-11-07,MTY,alejandro.aragon@sintec.com,AASA961107HNLRLL06
5,SINTEC,GABA960511HDFRRL06,M,Alejandro,Graf Burgos,1996-05-11,11,5,1996,1996-05-11,CDMX,alejandro.graf@sintec.com,GABA960511HDFRRL06
6,SINTEC,LOGA970321HNLPTL06,M,Alejandro,Lopez Gutierrez,1997-03-21,21,3,1997,1997-03-21,MTY,alejandro.lopez@sintec.com,LOGA970321HNLPTL06
7,SINTEC,MAMA000804HDFRLLA4,M,Alejandro,Martínez Molina,2000-08-04,4,8,2000,2000-08-04,CDMX,alejandro.martinez@sintec.com,MAMA000804HDFRLLA4
8,SINTEC,UICD980624HNLRRV06,M,David Alejandro,Uriarte Correa,1998-06-24,24,6,1998,1998-06-24,MTY,alejandro.uriarte@sintec.com,UICD980624HNLRRV06
9,SINTEC,17493133K,F,Alejandro Daniel,Zepeda Navea,1990-12-07,7,12,1990,1990-12-07,CHI,alejandro.zepeda@sintec.com,17493133K


In [232]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16072\2118328728.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16072\2118328728.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [233]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(198, 23)

In [234]:
Pacientes_con_id.head()

,empl,rut1,sexo,nombres,apellidos,nacimiento,gth,fghfjg,fj,fgjfgj,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,SINTEC,LUGA810606HDFNMB01,M,Abdon,Luna Gomez,1981-06-06,6,6,1981,1981-06-06,...,True,0018c00002iw09dAAA,LUGA810606HDFNMB01,Abdon Luna Gomez,CURP,None,None,Cdmx,Hombre,None
1,SINTEC,ROAA900309MVZDGD07,F,Adriana,Rodriguez Aguilar,1990-03-09,9,3,1990,1990-03-09,...,True,0018c00002iw09eAAA,ROAA900309MVZDGD07,Adriana Rodriguez Aguilar,CURP,None,None,Mty,Mujer,None
2,SINTEC,LOGA970503HNLZRL04,M,Alberto,Lozano Garza,1997-05-03,3,5,1997,1997-05-03,...,True,0018c00002iw09fAAA,LOGA970503HNLZRL04,Alberto Lozano Garza,CURP,None,None,Mty,Hombre,None
3,SINTEC,ZACA890613MDFMRL09,F,Alejandra Berenice,Zamora Cruz,1989-06-13,13,6,1989,1989-06-13,...,True,0018c00002iw09gAAA,ZACA890613MDFMRL09,Alejandra Berenice Zamora Cruz,CURP,None,None,Mty,Mujer,None
4,SINTEC,AASA961107HNLRLL06,M,Alejandro,Aragon Salazar,1996-11-07,7,11,1996,1996-11-07,...,True,0018c00002iw09hAAA,AASA961107HNLRLL06,Alejandro Aragon Salazar,CURP,None,None,Mty,Hombre,None
5,SINTEC,GABA960511HDFRRL06,M,Alejandro,Graf Burgos,1996-05-11,11,5,1996,1996-05-11,...,True,0018c00002iw09iAAA,GABA960511HDFRRL06,Alejandro Graf Burgos,CURP,None,None,Cdmx,Hombre,None
6,SINTEC,LOGA970321HNLPTL06,M,Alejandro,Lopez Gutierrez,1997-03-21,21,3,1997,1997-03-21,...,True,0018c00002iw09jAAA,LOGA970321HNLPTL06,Alejandro Lopez Gutierrez,CURP,None,None,Mty,Hombre,None
7,SINTEC,MAMA000804HDFRLLA4,M,Alejandro,Martínez Molina,2000-08-04,4,8,2000,2000-08-04,...,True,0018c00002iw09kAAA,MAMA000804HDFRLLA4,Alejandro Martínez Molina,CURP,None,None,Cdmx,Hombre,None
8,SINTEC,UICD980624HNLRRV06,M,David Alejandro,Uriarte Correa,1998-06-24,24,6,1998,1998-06-24,...,True,0018c00002iw09lAAA,UICD980624HNLRRV06,David Alejandro Uriarte Correa,CURP,None,None,Mty,Hombre,None
9,SINTEC,17493133K,F,Alejandro Daniel,Zepeda Navea,1990-12-07,7,12,1990,1990-12-07,...,True,0018c00002iw09mAAA,17493133K,Alejandro Daniel Zepeda Navea,CURP,None,None,Chi,Mujer,None


In [235]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [236]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Pacientes_con_id["rut"]
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]
if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        'Varón':'Hombre'      
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = Pacientes_con_id["celular"]

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"




if Pais == 'Chile':        
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtnEAE'
else:    
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtsEAE'


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,PersonDepartment,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
13,False,Amanda,Dal Magro De Lemos Paim,3158694026,amanda.paim@sintec.com,1995-05-04,Mujer,Sao,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE
14,False,Ana Maria,Alvarez Agudelo,1019134634,ana.alvarez@sintec.com,1997-10-28,Mujer,Bog,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE
18,False,Andrea,Ospina Ramirez,1010244820,andrea.ospina@sintec.com,1999-12-03,Mujer,Bog,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE
20,False,Andres Felipe,Peralta Davila,1015454780,andres.peralta@sintec.com,1995-06-04,Hombre,Bog,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE
21,False,Andres Felipe,Ramírez Giraldo,1036625719,andres.ramirez@sintec.com,1986-01-02,Hombre,Bog,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE
22,False,Andres Felipe,Rodriguez Velandia,1020831374,andres.rodriguez@sintec.com,1998-01-27,Hombre,Bog,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE
24,False,Angélica Esperanza,Corredor Rodríguez,52788928,angelica.corredor@sintec.com,1980-05-21,Mujer,Bog,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE
25,False,Antony Wilmar,Alzate Embus,1012405991,antony.alzate@sintec.com,1993-12-17,Hombre,Bog,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE
28,False,Arturo,Malagon Samperio,275443972,arturo.malagon@sintec.com,1982-02-19,Hombre,Chi,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE
36,False,Cristina,Damazio Pacheco,10841773904,cristina.damazio@sintec.com,1996-07-08,Mujer,Sao,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE


In [238]:
    
clinica_virtual = pd.DataFrame()    
clinica_virtual['rut'] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual['firstName'] = Pacientes_con_id['nombres'].str.title().str.strip()
clinica_virtual['lastName'] = Pacientes_con_id['apellidos'].str.title().str.strip()
clinica_virtual['accountId'] = campaña



Codigos_Clinica_Virtual = {
'BCI':'1',
'Farmacias Ahumada':'2',
'MetLife':'3',
'Vida Security':'4',
'Willis':'5',
'Entel':'6',
'Testing':'7',
'3SP':'8',
'HomeCare':'9',
'Pedro de Valdivia':'10',
'Colaboradores':'11',
'MineraAbra':'12',
'CMPC':'13',
'Lipigas':'14',
'Asismed':'15',
'Siempre Perú':'16',
'Farmacias Cruz Verde':'17',
'WFS Food Services':'18',
'VidaCel':'19',
'Dermik':'20',
'EY':'21',
'IND':'22',
'Sindicato Walmart':'23',
'Instruvalve':'24',
'VidaCel Colaboradores':'25',
'Socofar':'26',
'Milan Fajanovic':'27',
'THBseguros':'28',
'Pesco':'29',
'Burgos - Consorcio':'30',
'COFAMA-SILENTIUM':'31',
'KAUFMANN':'32',
'QuePlan':'33',
'Zurich':'34',
'Colaboradores Siempre Chile':'35',
'Talo':'36',
'Transvip':'37',
'Vida Security Colaboradores':'38',
'FORUS':'39',
'SURA':'40',
'Southbrige':'41',
'Construmart':'42',
'BICE Vida':'43'

}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [239]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A


In [240]:
campaña_seleccionada.shape

(0, 12)

In [241]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [242]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
